In [1]:
import requests
import pandas as pd

In [2]:
api_key = "df976c50384d9ee14627e6f9e477a6f47b59822c"

In [3]:
acs_base = "https://api.census.gov/data/2022/acs/acs1"

In [4]:
acs_vars = [
    "NAME", "B19013_001E",  # Median income
    "B15003_001E",          # Total education population (25+)
    "B15003_017E",          # High school grad
    "B15003_022E",          # Bachelor's
    "B15003_023E",          # Master's
    "B15003_025E"           # Doctorate
]

In [5]:
acs_url = f"{acs_base}?get={','.join(acs_vars)}&for=state:*&key={api_key}"
acs_response = requests.get(acs_url)
acs_data = acs_response.json()

In [6]:
acs_df = pd.DataFrame(acs_data[1:], columns=acs_data[0])
acs_df = acs_df.rename(columns={
    "NAME": "State",
    "B19013_001E": "Median_Income",
    "B15003_001E": "Edu_Total",
    "B15003_017E": "HS_Grad",
    "B15003_022E": "Bachelors",
    "B15003_023E": "Masters",
    "B15003_025E": "Doctorate"
})

In [7]:
acs_df[["Median_Income", "Edu_Total", "HS_Grad", "Bachelors", "Masters", "Doctorate"]] = \
    acs_df[["Median_Income", "Edu_Total", "HS_Grad", "Bachelors", "Masters", "Doctorate"]].astype(int)


In [8]:
acs_df.head()

,State,Median_Income,Edu_Total,HS_Grad,Bachelors,Masters,Doctorate,state
0,Alabama,59674,3474924,874286,609316,286677,42908,01
1,Alaska,88121,489218,116008,94168,39551,6548,02
2,Arizona,74568,5053656,996330,1032052,463372,73242,04
3,Arkansas,55432,2057624,583402,323961,144054,23987,05
4,California,91551,26866773,4846859,6056169,2643964,517699,06


In [9]:
# 2020 Census Urban/Rural Area - RUCC Data 

In [10]:
rucc_df = pd.read_csv('ruralurbancodes2023.csv', dtype={'FIPS': str}, encoding='latin1') # Use latin1 because  RUCC CSV file isn’t in UTF-8 encoding which is typical for government files
rucc_df.head()

,FIPS,State,County_Name,Attribute,Value
0,01001,AL,Autauga County,Population_2020,58805
1,01001,AL,Autauga County,RUCC_2023,2
2,01001,AL,Autauga County,Description,"Metro - Counties in metro areas of 250,000 to ..."
3,01003,AL,Baldwin County,Population_2020,231767
4,01003,AL,Baldwin County,RUCC_2023,3


In [13]:
# Filter for RUCC_2023 only and clean
rucc_clean = rucc_df[rucc_df["Attribute"] == "RUCC_2023"].copy()
rucc_clean["RUCC_2023"] = pd.to_numeric(rucc_clean["Value"], errors="coerce")
rucc_clean = rucc_clean.dropna(subset=["RUCC_2023"])
rucc_clean["RUCC_2023"] = rucc_clean["RUCC_2023"].astype(int)


In [14]:
def label_urban_rural(code):
    if code in [1, 2, 3]:
        return "Urban"
    else:
        return "Rural"

rucc_clean["RUCC_Label"] = rucc_clean["RUCC_2023"].apply(label_urban_rural)

# Extract state FIPS
rucc_clean["state"] = rucc_clean["FIPS"].str[:2]


In [15]:
# Group and count
state_counts = rucc_clean.groupby(["state", "RUCC_Label"]).size().unstack(fill_value=0)
state_counts["Total"] = state_counts.sum(axis=1)

# Percentages
for col in ["Urban", "Rural"]:
    if col in state_counts.columns:
        state_counts[f"{col}_Pct"] = (state_counts[col] / state_counts["Total"]) * 100
    else:
        state_counts[f"{col}_Pct"] = 0

# Classify states
def classify_state(row):
    if row["Urban_Pct"] >= 80:
        return "Urban"
    elif row["Rural_Pct"] >= 50:
        return "Rural"
    else:
        return "Mixed"

state_counts["State_Type"] = state_counts.apply(classify_state, axis=1)

# Final formatting
state_counts = state_counts.reset_index()
state_counts["state"] = state_counts["state"].astype(str).str.zfill(2)


In [16]:
acs_df["state"] = acs_df["state"].astype(str).str.zfill(2)
merged_df = pd.merge(acs_df, state_counts, on="state", how="left")


In [ ]:
merged_df.head()

,State,Median_Income,Edu_Total,HS_Grad,Bachelors,Masters,Doctorate,state,Rural,Urban,Total,Urban_Pct,Rural_Pct,State_Type
0,Alabama,59674,3474924,874286,609316,286677,42908,01,36,31,67,46.268657,53.731343,Rural
1,Alaska,88121,489218,116008,94168,39551,6548,02,27,3,30,10.000000,90.000000,Rural
2,Arizona,74568,5053656,996330,1032052,463372,73242,04,7,8,15,53.333333,46.666667,Mixed
3,Arkansas,55432,2057624,583402,323961,144054,23987,05,58,17,75,22.666667,77.333333,Rural
4,California,91551,26866773,4846859,6056169,2643964,517699,06,21,37,58,63.793103,36.206897,Mixed
